## Homework: Vector Search

In this homework, we will learn more about vector search
and embedding. Like in the module, we will use Qdrant and
fastembed

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

Note: if you want to learn how vector search works under
the hood, check 
[homework 3 from 2024 cohort (questions 1-4)](../../2024/03-vector-search/homework.md)

## Embeddings

Qdrant uses [fastembed](https://github.com/qdrant/fastembed)
under the hood to turn text into vectors. 

We will now explore this library

Make sure it's installed:

```bash
pip install fastembed
```

Import it: 

In [1]:
from fastembed import TextEmbedding

## Q1. Embedding the query

Embed the query: `'I just discovered the course. Can I join now?'`.
Use the `'jinaai/jina-embeddings-v2-small-en'` model. 

You should get a numpy array of size 512.

What's the minimal value in this array?

* -0.51
* `-0.11`
* 0
* 0.51

In [ ]:
# embedding model and embedding dimensions to use
model_handle = "jinaai/jina-embeddings-v2-small-en"
EMBEDDING_DIMENSIONALITY = 512
embedding_model = TextEmbedding(model_handle)

text = "I just discovered the course. Can I join now?"

embedded_text = list(embedding_model.embed(documents=text))
min(embedded_text[0])

np.float64(-0.11726373551188797)

The minimal value in the array is -0.11


## Cosine similarity

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can chech that by using the `norm` function:

```python
import numpy as np
np.linalg.norm(q)
```

Which means that we can simply compute the dot product between
two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

```python
q.dot(q)
```

In [ ]:
import numpy as np

q = embedded_text[0]
np.linalg.norm(q), q.dot(q)

(np.float64(1.0), np.float64(1.0))

## Q2. Cosine similarity with another vector

Now let's embed this document:

```python
doc = 'Can I still join the course after the start date?'
```

What's the cosine similarity between the vector for the query
and the vector for the document?

* 0.3
* 0.5
* 0.7
* `0.9`

In [15]:
doc = 'Can I still join the course after the start date?'
embedded_text = list(embedding_model.embed(documents=doc))
q2 = embedded_text[0]

# calculate cosine similarity
q.dot(q2)

np.float64(0.9008528856818037)

The cosine similarity is 0.9.

## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

In [16]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

Compute the embeddings for the text field, and compute the 
cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

- 0
- `1`
- 2
- 3
- 4

Hint: if you put all the embeddings of the text field in one matrix `V` (a single 2-dimensional numpy array), then
computing the cosine becomes a matrix multiplication:

```python
V.dot(q)
```

If this hint is rather confusing you than helping, feel free
to ignore it.

In [23]:
# compute the embedding matrix
embedded_mat = []
for doc in documents:
    embedded_vector = list(embedding_model.embed(documents=doc["text"]))
    embedded_mat.append(embedded_vector[0])

embedded_mat = np.stack(embedded_mat)

cosine_similarities = embedded_mat.dot(q)
np.argmax(cosine_similarities), cosine_similarities

(np.int64(1),
 array([0.76296845, 0.81823783, 0.80853973, 0.71330786, 0.73044994]))

Document index 1 has the highest similarity.

## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of
`question` and `text`:

```python
full_text = doc['question'] + ' ' + doc['text']
``` 

Embed this field and compute the cosine between it and the
query vector. What's the highest scoring document?

- `0`
- 1
- 2
- 3
- 4

Is it different from Q3? If yes, why?

In [24]:
# compute the embedding matrix
embedded_mat = []
for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    embedded_vector = list(embedding_model.embed(documents=full_text))
    embedded_mat.append(embedded_vector[0])

embedded_mat = np.stack(embedded_mat)

cosine_similarities = embedded_mat.dot(q)
np.argmax(cosine_similarities), cosine_similarities

(np.int64(0),
 array([0.85145431, 0.84365941, 0.84082872, 0.77551577, 0.8086008 ]))

The highest scoring document is document 0, which is different from previous question. I believe the reason is that `question` is a better field when we search with the query vector, and for this question, we concatenate `text` and `question` together so the search result is improved. 

## Q5. Selecting the embedding model

Now let's select a smaller embedding model.
What's the smallest dimensionality for models in fastembed?

- 128
- 256
- `384`
- 512

One of these models is `BAAI/bge-small-en`. Let's use it.

In [36]:
models = TextEmbedding.list_supported_models()
smallest_dim_model = min(models, key=lambda x: x['dim'])
smallest_dim_model, smallest_dim_model['dim']

({'model': 'BAAI/bge-small-en',
  'sources': {'hf': 'Qdrant/bge-small-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.13,
  'additional_files': [],
  'dim': 384,
  'tasks': {}},
 384)

The smallest dimensionality for models in fastembed is 384 from `BAAI/bge-small-en`

## Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

In [37]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

```python
text = doc['question'] + ' ' + doc['text']
```


After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results?
(The score for the first returned record):

- 0.97
- `0.87`
- 0.77
- 0.67


In [41]:
# create qdrant client
from qdrant_client import QdrantClient, models
qdrant_client = QdrantClient(url='http://localhost:6333')

# embedding model and embedding dimensions to use
model_handle = "BAAI/bge-small-en"
EMBEDDING_DIMENSIONALITY = 384

In [43]:
# create collection
collection_name = "zoomcamp-hw2"

qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [44]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp'}

In [45]:
points = []
for i, doc in enumerate(documents):
    full_text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=i,
        vector=models.Document(text=full_text, model=model_handle),
        payload={
            "full_text": full_text,
            "section": doc['section'],
            "course": doc["course"]
        }
    )
    points.append(point)

In [46]:
qdrant_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [49]:
# search
def search(query, limit=1):

    results = qdrant_client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

query = 'I just discovered the course. Can I join now?'
result = search(query)
result.points[0], result.points[0].score

(ScoredPoint(id=14, version=0, score=0.8703172, payload={'full_text': 'The course has already started. Can I still join it? Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None),
 0.8703172)

The highest score in the results is 0.87.

## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2025/homework/hw2
* It's possible that your answers won't match exactly. If it's the case, select the closest one.